In [ ]:
import cv2
import numpy as np

def extract_connected_regions(image, points, connectivity=4):
    """
    Given an edge-detected image and a list of seed points,
    returns a mask for each connected region that shares the same color
    as the seed point (using flood fill or connected components).

    Args:
        image (np.ndarray): Input image (grayscale or binary).
        points (list of tuples): List of (x, y) seed points.
        connectivity (int): 4 or 8 connectivity for region growing.

    Returns:
        regions (list of np.ndarray): List of binary masks, one per point.
    """
    # Ensure grayscale
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    h, w = gray.shape
    regions = []

    # Keep track of processed pixels to avoid overlaps
    processed = np.zeros_like(gray, dtype=np.uint8)

    for i, (x, y) in enumerate(points):
        if x < 0 or y < 0 or x >= w or y >= h:
            print(f"Skipping invalid point {x, y}")
            continue

        target_value = gray[y, x]

        # Skip if already processed
        if processed[y, x]:
            continue

        # Flood fill to get connected region
        mask = np.zeros((h + 2, w + 2), np.uint8)  # +2 for OpenCV floodFill border
        _, _, _, rect = cv2.floodFill(
            gray.copy(), mask, (x, y),
            newVal=255,  # just temporary
            loDiff=0, upDiff=0,
            flags=connectivity | (255 << 8)
        )

        # Flood-filled mask (remove 1px border added by OpenCV)
        region_mask = (mask[1:-1, 1:-1] > 0).astype(np.uint8)

        # Update processed
        processed = cv2.bitwise_or(processed, region_mask)

        regions.append(region_mask)

    return regions

In [ ]:
# Load a binary edge image
img = cv2.imread("edges.png", cv2.IMREAD_GRAYSCALE)

# List of seed points (x, y)
points = [(50, 60), (200, 120), (340, 400)]

# Get connected regions
regions = extract_connected_regions(img, points)

dis